In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

class NeuralNetwork:
    def __init__(self, input_size, hidden_layers, output_size, activation='relu', learning_rate=0.01, problem_type='regression'):
        self.layers = [input_size] + hidden_layers + [output_size]
        self.activation = activation
        self.learning_rate = learning_rate
        self.problem_type = problem_type
        self.weights = []
        self.biases = []
        self.history = {}

        for i in range(len(self.layers) - 1):
            w = np.random.randn(self.layers[i], self.layers[i+1]) * (np.sqrt(2.0 / self.layers[i]) if activation == 'relu' else np.sqrt(1.0 / self.layers[i]))
            self.weights.append(w)
            self.biases.append(np.zeros((1, self.layers[i+1])))

    def _activation_function(self, x, derivative=False):
        if self.activation == 'sigmoid':
            sig = 1 / (1 + np.exp(-np.clip(x, -500, 500)))
            return sig * (1 - sig) if derivative else sig
        elif self.activation == 'relu':
            return (x > 0).astype(float) if derivative else np.maximum(0, x)
        elif self.activation == 'tanh':
            return 1 - np.tanh(x)**2 if derivative else np.tanh(x)
        elif self.activation == 'leaky_relu':
            return np.where(x > 0, 1, 0.01) if derivative else np.where(x > 0, x, 0.01 * x)

    def _softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def forward(self, X):
        self.z_values = []
        self.activations = [X]

        for i in range(len(self.weights)):
            z = np.dot(self.activations[-1], self.weights[i]) + self.biases[i]
            self.z_values.append(z)

            if i == len(self.weights) - 1:
                a = self._softmax(z) if self.problem_type == 'classification' and self.layers[-1] > 1 else (self._activation_function(z) if self.problem_type == 'classification' else z)
            else:
                a = self._activation_function(z)
            self.activations.append(a)
        return self.activations[-1]

    def backward(self, X, y, output):
        m = X.shape[0]

        if self.problem_type == 'classification' and self.layers[-1] > 1:
            y_onehot = np.zeros((m, self.layers[-1]))
            y_onehot[np.arange(m), y.astype(int)] = 1
            delta = output - y_onehot
        else:
            y_reshaped = y.reshape(-1, 1) if len(y.shape) == 1 else y
            delta = output - y_reshaped

        for i in reversed(range(len(self.weights))):
            self.weights[i] -= self.learning_rate * np.dot(self.activations[i].T, delta) / m
            self.biases[i] -= self.learning_rate * np.sum(delta, axis=0, keepdims=True) / m
            if i > 0:
                delta = np.dot(delta, self.weights[i].T) * self._activation_function(self.z_values[i-1], derivative=True)

    def train(self, X, y, epochs=1000, verbose=False):
        losses, accuracies = [], []

        for epoch in range(epochs):
            output = self.forward(X)

            if self.problem_type == 'classification' and self.layers[-1] > 1:
                y_onehot = np.zeros((len(y), self.layers[-1]))
                y_onehot[np.arange(len(y)), y.astype(int)] = 1
                loss = -np.mean(np.sum(y_onehot * np.log(output + 1e-15), axis=1))
                accuracy = np.mean(np.argmax(output, axis=1) == y)
                accuracies.append(accuracy)
            else:
                y_reshaped = y.reshape(-1, 1) if len(y.shape) == 1 else y
                loss = np.mean((output - y_reshaped)**2)
                accuracies.append(0)

            losses.append(loss)
            self.backward(X, y, output)

            if verbose and epoch % 100 == 0:
                print(f'Epoch {epoch}, Loss: {loss:.4f}' + (f', Accuracy: {accuracy:.4f}' if self.problem_type == 'classification' else ''))

        self.history = {'loss': losses, 'accuracy': accuracies}
        return losses

    def predict(self, X):
        output = self.forward(X)
        return np.argmax(output, axis=1) if self.problem_type == 'classification' and self.layers[-1] > 1 else output

class DataProcessor:
    def __init__(self):
        self.encoders = {}
        self.scalers = {}

    def load_and_prepare_data(self, filepath='train_data.csv'):
        print("Ładowanie i przygotowanie danych...")
        df = pd.read_csv(filepath, sep=';')
        print(f"Rozmiar danych: {df.shape}")
        print(f"Kolumny: {df.columns.tolist()}")

        df = df.drop(['case_id', 'patientid'], axis=1, errors='ignore')

        categorical_columns = ['Hospital_type_code', 'Department', 'Ward_Type', 'Ward_Facility_Code',
                              'Type of Admission', 'Severity of Illness', 'Age', 'Stay', 'Hospital_region_code']

        for col in categorical_columns:
            if col in df.columns:
                le = LabelEncoder()
                df[col] = le.fit_transform(df[col].astype(str))
                self.encoders[col] = le

        df = df.fillna(df.mean())
        print(f"Liczba kolumn po przetworzeniu: {df.shape[1]}")
        return df

    def prepare_regression_data(self, df, target_col='Admission_Deposit'):
        X = df.drop([target_col], axis=1).values
        y = df[target_col].values

        self.scalers['X_reg'] = StandardScaler()
        self.scalers['y_reg'] = StandardScaler()

        X_scaled = self.scalers['X_reg'].fit_transform(X)
        y_scaled = self.scalers['y_reg'].fit_transform(y.reshape(-1, 1)).flatten()

        return train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

    def prepare_classification_data(self, df, target_col='Stay'):
        X = df.drop([target_col], axis=1).values
        y_original = df[target_col].values

        def categorize_stay(stay_encoded):
            stay_original = self.encoders[target_col].inverse_transform([stay_encoded])[0]
            return 0 if '0-10' in stay_original else 1 if '11-20' in stay_original else 2 if '21-30' in stay_original else 3 if '31-40' in stay_original else 4

        y = np.array([categorize_stay(stay) for stay in y_original])

        self.scalers['X_class'] = StandardScaler()
        X_scaled = self.scalers['X_class'].fit_transform(X)

        print(f"Rozkład klas: {np.bincount(y)}")
        print("Kategorie: 0=0-10dni, 1=11-20dni, 2=21-30dni, 3=31-40dni, 4=>40dni")

        return train_test_split(X_scaled, y, test_size=0.2, random_state=42)

class ModelEvaluator:
    @staticmethod
    def test_parameters(X_train, X_test, y_train, y_test, param_name, param_values, base_params, problem_type='regression', n_trials=3):
        results = []
        print(f"\nTestowanie parametru: {param_name}")

        for param_value in param_values:
            print(f"  Testowanie {param_name} = {param_value}")
            train_scores, test_scores = [], []

            for trial in range(n_trials):
                params = base_params.copy()
                params[param_name] = param_value

                nn = NeuralNetwork(
                    input_size=X_train.shape[1],
                    hidden_layers=params.get('hidden_layers', [10]),
                    output_size=5 if problem_type == 'classification' else 1,
                    activation=params.get('activation', 'relu'),
                    learning_rate=params.get('learning_rate', 0.01),
                    problem_type=problem_type
                )

                nn.train(X_train, y_train, epochs=params.get('epochs', 500))

                train_pred = nn.predict(X_train)
                test_pred = nn.predict(X_test)

                if problem_type == 'regression':
                    train_pred = train_pred.flatten()
                    test_pred = test_pred.flatten()
                    train_score = np.sqrt(mean_squared_error(y_train, train_pred))
                    test_score = np.sqrt(mean_squared_error(y_test, test_pred))
                else:
                    train_score = accuracy_score(y_train, train_pred)
                    test_score = accuracy_score(y_test, test_pred)

                train_scores.append(train_score)
                test_scores.append(test_score)

            results.append({
                'parameter': param_name,
                'value': str(param_value),
                'train_mean': np.mean(train_scores),
                'train_std': np.std(train_scores),
                'train_best': np.min(train_scores) if problem_type == 'regression' else np.max(train_scores),
                'test_mean': np.mean(test_scores),
                'test_std': np.std(test_scores),
                'test_best': np.min(test_scores) if problem_type == 'regression' else np.max(test_scores)
            })

        return results

    @staticmethod
    def visualize_best_model(X_train, X_test, y_train, y_test, best_params, problem_type='classification'):
        print(f"\nTrenowanie najlepszego modelu z parametrami: {best_params}")

        nn = NeuralNetwork(
            input_size=X_train.shape[1],
            hidden_layers=best_params['hidden_layers'],
            output_size=5 if problem_type == 'classification' else 1,
            activation=best_params['activation'],
            learning_rate=best_params['learning_rate'],
            problem_type=problem_type
        )

        nn.train(X_train, y_train, epochs=best_params['epochs'], verbose=True)

        y_pred_test = nn.predict(X_test)
        y_pred_train = nn.predict(X_train)

        if problem_type == 'classification':
            class_names = ['0-10 dni', '11-20 dni', '21-30 dni', '31-40 dni', '>40 dni']
            train_acc = accuracy_score(y_train, y_pred_train)
            test_acc = accuracy_score(y_test, y_pred_test)
            print(f"\nWyniki najlepszego modelu:")
            print(f"Accuracy na zbiorze treningowym: {train_acc:.4f}")
            print(f"Accuracy na zbiorze testowym: {test_acc:.4f}")

            plt.figure(figsize=(15, 5))

            plt.subplot(1, 3, 1)
            cm = confusion_matrix(y_test, y_pred_test)
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
            plt.title('Macierz pomyłek - Najlepszy model')
            plt.xlabel('Przewidywane')
            plt.ylabel('Rzeczywiste')

            plt.subplot(1, 3, 2)
            plt.plot(nn.history['loss'])
            plt.title('Loss podczas treningu')
            plt.xlabel('Epoka')
            plt.ylabel('Loss')
            plt.grid(True)

            plt.subplot(1, 3, 3)
            plt.plot(nn.history['accuracy'])
            plt.title('Accuracy podczas treningu')
            plt.xlabel('Epoka')
            plt.ylabel('Accuracy')
            plt.grid(True)

            plt.tight_layout()
            plt.show()

            print("\nRaport klasyfikacji:")
            print(classification_report(y_test, y_pred_test, target_names=class_names))
            return nn, test_acc

        else:
            y_pred_test = y_pred_test.flatten()
            y_pred_train = y_pred_train.flatten()
            train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
            test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
            print(f"\nWyniki najlepszego modelu:")
            print(f"RMSE na zbiorze treningowym: {train_rmse:.4f}")
            print(f"RMSE na zbiorze testowym: {test_rmse:.4f}")

            plt.figure(figsize=(12, 4))

            plt.subplot(1, 2, 1)
            plt.scatter(y_test, y_pred_test, alpha=0.5)
            plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
            plt.xlabel('Rzeczywiste wartości')
            plt.ylabel('Przewidywane wartości')
            plt.title('Predykcje vs Rzeczywiste wartości (Test)')
            plt.grid(True)

            plt.subplot(1, 2, 2)
            plt.plot(nn.history['loss'])
            plt.title('Loss podczas treningu')
            plt.xlabel('Epoka')
            plt.ylabel('Loss (MSE)')
            plt.grid(True)

            plt.tight_layout()
            plt.show()
            return nn, test_rmse

    @staticmethod
    def analyze_results(results, problem_type='classification'):
        print(f"\n{'='*80}")
        print(f"ANALIZA WYNIKÓW - {problem_type.upper()}")
        print(f"{'='*80}")

        best_result = max(results, key=lambda x: x['test_best']) if problem_type == 'classification' else min(results, key=lambda x: x['test_best'])
        metric_name = "Accuracy" if problem_type == 'classification' else "RMSE"

        print(f"\nNAJLEPSZY WYNIK:")
        print(f"{metric_name} = {best_result['test_best']:.4f}")
        print(f"Parametr: {best_result['parameter']} = {best_result['value']}")
        print(f"Średnia ± std: {best_result['test_mean']:.4f} ± {best_result['test_std']:.4f}")

        param_groups = {}
        for result in results:
            param = result['parameter']
            if param not in param_groups:
                param_groups[param] = []
            param_groups[param].append(result)

        for param_name, param_results in param_groups.items():
            print(f"\n{param_name.upper()}:")
            best_param = max(param_results, key=lambda x: x['test_best']) if problem_type == 'classification' else min(param_results, key=lambda x: x['test_best'])
            print(f"  Najlepszy: {best_param['value']} ({metric_name}: {best_param['test_best']:.4f})")

        return best_result

def main():
    processor = DataProcessor()
    evaluator = ModelEvaluator()
    df = processor.load_and_prepare_data()

    test_params = {
        'hidden_layers': [[5], [10], [20], [30], [50], [10, 5], [20, 10], [30, 15], [20, 10, 5], [30, 20, 10], [50, 25, 10]],
        'activation': ['sigmoid', 'relu', 'tanh', 'leaky_relu'],
        'learning_rate': [0.001, 0.01, 0.1, 0.2]
    }

    X_train_reg, X_test_reg, y_train_reg, y_test_reg = processor.prepare_regression_data(df)
    all_results_reg = []
    for param_name, param_values in test_params.items():
        all_results_reg.extend(evaluator.test_parameters(X_train_reg, X_test_reg, y_train_reg, y_test_reg, param_name, param_values, {'hidden_layers': [10], 'activation': 'relu', 'learning_rate': 0.01, 'epochs': 500}, 'regression'))

    X_train_class, X_test_class, y_train_class, y_test_class = processor.prepare_classification_data(df)
    all_results_class = []
    for param_name, param_values in test_params.items():
        all_results_class.extend(evaluator.test_parameters(X_train_class, X_test_class, y_train_class, y_test_class, param_name, param_values, {'hidden_layers': [20, 10], 'activation': 'relu', 'learning_rate': 0.01, 'epochs': 1000}, 'classification'))

    for problem_type, results in [("REGRESYJNY", all_results_reg), ("KLASYFIKACYJNY", all_results_class)]:
        print(f"\nPROBLEM {problem_type}:")
        for result in results:
            print(f"{result['parameter']:<15} {result['value']:<20} {result['train_mean']:.4f}±{result['train_std']:.3f} {result['test_mean']:.4f}±{result['test_std']:.3f} {result['test_best']:.4f}")

    best_reg = evaluator.analyze_results(all_results_reg, 'regression')
    best_class = evaluator.analyze_results(all_results_class, 'classification')

    get_best = lambda results, param, is_reg: (min if is_reg else max)(r for r in results if r['parameter'] == param)

    best_arch_reg = get_best(all_results_reg, 'hidden_layers', True)
    best_arch_class = get_best(all_results_class, 'hidden_layers', False)
    best_act_reg = get_best(all_results_reg, 'activation', True)
    best_act_class = get_best(all_results_class, 'activation', False)
    best_lr_reg = get_best(all_results_reg, 'learning_rate', True)
    best_lr_class = get_best(all_results_class, 'learning_rate', False)

    best_params_reg = {'hidden_layers': eval(best_arch_reg['value']), 'activation': best_act_reg['value'], 'learning_rate': float(best_lr_reg['value']), 'epochs': 1000}
    best_params_class = {'hidden_layers': eval(best_arch_class['value']), 'activation': best_act_class['value'], 'learning_rate': float(best_lr_class['value']), 'epochs': 1500}

    final_rmse = evaluator.visualize_best_model(X_train_reg, X_test_reg, y_train_reg, y_test_reg, best_params_reg, 'regression')[1]
    final_acc = evaluator.visualize_best_model(X_train_class, X_test_class, y_train_class, y_test_class, best_params_class, 'classification')[1]

    print(f"\nNajlepszy model regresyjny - RMSE: {final_rmse:.4f}")
    print(f"Najlepszy model klasyfikacyjny - Accuracy: {final_acc:.4f}")

if __name__ == "__main__":
    main()

Ładowanie i przygotowanie danych...
Rozmiar danych: (318438, 18)
Kolumny: ['case_id', 'Hospital_code', 'Hospital_type_code', 'City_Code_Hospital', 'Hospital_region_code', 'Available Extra Rooms in Hospital', 'Department', 'Ward_Type', 'Ward_Facility_Code', 'Bed Grade', 'patientid', 'City_Code_Patient', 'Type of Admission', 'Severity of Illness', 'Visitors with Patient', 'Age', 'Admission_Deposit', 'Stay']
Liczba kolumn po przetworzeniu: 16

Testowanie parametru: hidden_layers
  Testowanie hidden_layers = [5]
  Testowanie hidden_layers = [10]
